In [ ]:
import pandas as pd
import warnings
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as scint
import sys
import time
warnings.filterwarnings("ignore")

In [ ]:
def run_bee_eq(x, ext_temp, truth, peri, treat):
    def bee_eq(y, t, w1, w2, external_temperature, truth, peri, treat):
        hive_temperature = y
        u1 = [0]*len(hive_temperature)
        u2 = [0]*len(hive_temperature)
        for i in range(len(hive_temperature)):
            if external_temperature[i] >= 0:
                u1[i] = 1
            else:
                u2[i] = -1
        #external_temperature =  amplitude_temp_ext* np.sin(om*t + phase)  
        if treat == 0:
            dydt = -w1*hive_temperature*u1 + w2*hive_temperature*u2 + 2*(-hive_temperature + external_temperature)
        else:
            dydt = -w1*hive_temperature*u1 + w2*hive_temperature*u2 + 2*(-hive_temperature) + peri + external_temperature
        return dydt

    t_max = len(ext_temp)-1
    t = np.linspace(0,t_max, num = t_max+1)
    errors = []
    params = []
    signals = []
    w1 = np.linspace(0,100, num = 101)
    w2 = np.linspace(0,100, num = 101)
    yy_param = np.linspace(31,38,num=8)
    a = 10*np.ones(len(ext_temp))
    for i in w1:
        for j in w2:
            for k in yy_param:
                truth_ = truth - k
                ext_temp_ = ext_temp - k
                peri_ = peri - k
                sol = scint.odeint(bee_eq, a, t, args=(i,j, ext_temp_, truth_, peri_, treat), col_deriv = True, rtol = 1e-3, atol = 1e-3) # 'w' and 'amplitude_temp_ext'
                theta_t = sol[-1,:]
                errors.append(mean_absolute_error(truth_, theta_t))
                params.append([i,j,k,(truth_ - theta_t),theta_t])
                signals.append(theta_t)

    if errors == []:
        return [], [], []
    error_arg = np.argmin(errors)
    error_min = errors[error_arg]
    sig_min = signals[error_arg]
    param_min = params[error_arg]
    return error_min, param_min, sig_min

In [ ]:
def main(*argv):
    filename = argv[0]
    temp_data = argv[1]
    ext_data = argv[2]
    peri_data = argv[3]
    type_ = argv[4]
    
    data = pd.read_csv(filename)
    d = pd.DataFrame(data).groupby(data['Date'])
    
    time_ = []
    start = time.time()
    w_, e_, sig_ = [], [], []
    for i,j in d:
        x = np.linspace(0, len(j[temp_data].values)-1, num = len(j[temp_data]))
        j.reset_index()
        print(j['Date'].values[0])
        xx =j[ext_data].values
        yy = j[temp_data].values
        if type_ == 'Y':
            zz = j[peri_data].values
            treat = j['Treatment'].values[0]
        else:
            zz = j[temp_data].values
            treat = 0
        err, w, sig = run_bee_eq(x, xx , yy, zz, treat)
        w_.append(w)
        e_.append(err)
        sig_.append(sig)
        end = time.time()
        time_.append(np.round((end - start)/60,4))

In [4]:
if __name__ == "__main__":
    hive_number = input('Enter Hive Number: ')
    hive_type = input('Is it treated? Y or N: ')
    filename = 'C_'+hive_number+'.csv'
    temp_data = 'Temp_'+ hive_number
    ext_data = 'Temp_Box'
    peri_data = 'Temp_Easy_'+ hive_number
    main(filename, temp_data, ext_data, peri_data, hive_type)

Enter Hive Number: 265
Is it treated? Y or N: N


NameError: name 'main' is not defined